In [2]:
!pip install emoji

     |████████████████████████████████| 170 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=c05fed053f3ff4c109a537db01356f090b90a9048c898b8b5123699abae8f04b
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [10]:
import re
import numpy as np
import pandas as pd
import emoji
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [15]:
#extract the time
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = re.match(pattern, s)

    if result:
        return True
    return False

In [16]:
# find Authors or Contacts
def find_author(s):
    s = s.split(':')

    if len(s) == 2:
        return True
    else:
        return False

In [17]:
# Finding Messages
def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author= None
    return date, time, author, message

In [18]:

data = []
conversation = 'WhatsApp Chat with Manasi.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDatapoint(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [21]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [23]:
df = pd.DataFrame(data, columns=['Date', 'Time', 'Author', 'Message'])

df['Date'] = pd.to_datetime(df['Date'])

data = df.dropna()

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiments = SentimentIntensityAnalyzer()

data['Positive'] = [sentiments.polarity_scores(i)['pos'] for i in data['Message']]
data['Negative'] = [sentiments.polarity_scores(i)['neg'] for i in data['Message']]
data['Neutral'] = [sentiments.polarity_scores(i)['neu'] for i in data['Message']]
data.head()

,Date,Time,Author,Message,Positive,Negative,Neutral
0,2020-11-24,7:55 PM,Manasi😊,Hey mansi here,0.0,0.0,1.0
1,2020-11-24,7:55 PM,Abhi,Silver shades here 😂😂,0.0,0.0,1.0
2,2020-11-24,7:56 PM,Manasi😊,I know your name is Abhishikth,0.0,0.0,1.0
3,2020-11-24,7:56 PM,Manasi😊,😂😂,0.0,0.0,1.0
4,2020-11-24,7:56 PM,Abhi,Chalo sahii hai 😁😁,0.0,0.0,1.0


In [24]:
x = sum(data['Positive'])
y = sum(data['Negative'])
z = sum(data['Neutral'])

In [26]:
def sentiment_score(a, b, c):
    if(a > b) and (a > c):
        print('Positive !')
    elif (b > a) and (b > c):
        print('Negative !')
    else:
        print('Neutral !')
sentiment_score(x, y, z)

Neutral !
